In [ ]:
import json
import os
import time
from google.colab import files
import zipfile
from IPython.display import clear_output

# Installation et import des dépendances
try:
    !pip install yake --quiet
    import yake
    print("✅ YAKE installé avec succès")
except Exception as e:
    print(f"❌ Erreur d'installation : {str(e)}")
    raise

def show_progress(message):
    """Affiche un message de progression"""
    clear_output(wait=True)
    print(f"⏳ {message}...")

def upload_files():
    """Gère l'upload des fichiers avec suivi"""
    show_progress("Prêt à recevoir les fichiers")
    uploaded = files.upload()
    while not uploaded:
        show_progress("En attente de fichiers...")
        time.sleep(1)
        uploaded = files.upload()

    input_folder = '/content/input_files'
    os.makedirs(input_folder, exist_ok=True)

    for filename, content in uploaded.items():
        with open(os.path.join(input_folder, filename), 'wb') as f:
            f.write(content)
        show_progress(f"Fichier {filename} reçu")

    return input_folder

def process_files(input_folder):
    """Traite les fichiers avec suivi"""
    output_folder = '/content/output'
    os.makedirs(output_folder, exist_ok=True)

    file_count = len([f for f in os.listdir(input_folder) if f.endswith('.json')])
    processed = 0

    for filename in os.listdir(input_folder):
        if filename.endswith('.json'):
            try:
                show_progress(f"Traitement {processed+1}/{file_count} - {filename}")

                # Lecture
                with open(os.path.join(input_folder, filename), 'r', encoding='utf-8') as f:
                    data = json.load(f)

                # Traitement
                if isinstance(data, list):
                    for item in data:
                        process_item(item)
                else:
                    process_item(data)

                # Sauvegarde
                output_path = os.path.join(output_folder, filename)
                with open(output_path, 'w', encoding='utf-8') as f:
                    json.dump(data, f, indent=4, ensure_ascii=False)

                processed += 1

            except Exception as e:
                print(f"⚠️ Erreur sur {filename}: {str(e)}")
                continue

    return output_folder

def process_item(item):
    """Traite un élément individuel"""
    text = ""
    if item.get("type") == "video":
        text = item.get("transcription", "")
    elif item.get("type") == "article":
        text = item.get("contenu", "")
    elif item.get("type") == "course":
        text = "\n".join(s.get("section", "") for s in item.get("content", []))

    if text:
        kw_extractor = yake.KeywordExtractor(lan="fr" if "é" in text else "en", top=10)
        item["keywords"] = [kw[0] for kw in kw_extractor.extract_keywords(text)]

def create_zip(output_folder):
    """Crée une archive ZIP"""
    zip_path = '/content/resultats.zip'
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for root, _, files in os.walk(output_folder):
            for file in files:
                zipf.write(os.path.join(root, file), file)
    return zip_path

def main():
    try:
        # Étape 1: Upload
        print("📤 Veuillez sélectionner vos fichiers JSON")
        input_folder = upload_files()

        # Étape 2: Traitement
        output_folder = process_files(input_folder)

        # Étape 3: Préparation des résultats
        show_progress("Préparation du téléchargement")
        zip_path = create_zip(output_folder)

        # Étape 4: Téléchargement
        print("✅ Traitement terminé avec succès!")
        print("⬇️ Le téléchargement va commencer...")
        files.download(zip_path)

    except Exception as e:
        print(f"❌ Erreur critique : {str(e)}")

if __name__ == "__main__":
    main()

⏳ Préparation du téléchargement...
✅ Traitement terminé avec succès!
⬇️ Le téléchargement va commencer...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>